In [ ]:
import os
os.environ["HF_HOME"] = r"./.cache"

from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, \
    GenerationConfig, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model, TaskType

In [ ]:
SOURCE_LANG = "en"
DATASET_NAME = "mixed-500k"
RESUME = False


model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50")
TARGET_LANG = "ja" if SOURCE_LANG == "en" else "en"
if SOURCE_LANG == "en":
    tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50", src_lang="en_XX", tgt_lang="ja_XX")
else:
    tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50", src_lang="ja_XX", tgt_lang="en_XX")

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_count, trainable_bytes = 0, 0
    total_count, total_bytes = 0, 0
    for _, param in model.named_parameters():
        total_count += param.nelement()
        total_bytes += param.nelement() * param.element_size()
        if param.requires_grad:
            trainable_count += param.nelement()
            trainable_bytes += param.nelement() * param.element_size()
    print(
        f"Total params: {total_count:12,} ({(total_bytes / 1024**2):7,.1f}MB) | "
        f"Trainable params: {trainable_count:12,} ({(trainable_bytes / 1024**2):7,.1f}MB) [{100 * trainable_count / total_count:3.1f}%]"
    )

In [ ]:
modules_to_save = ["final_layer_norm", "self_attn_layer_norm", "layer_norm", "layernorm_embedding", "embed_positions"]
target_modules = ["q_proj", "k_proj", "v_proj", "out_proj", "fc1", "fc2", "shared", "lm_head"]
config = LoraConfig(
    r=8,
    lora_alpha=8, # anything is fine (simply tune lr)
    lora_dropout=0.1,
    target_modules=target_modules,
    modules_to_save=modules_to_save,
    bias="lora_only",
    task_type=TaskType.SEQ_2_SEQ_LM,
)
lora_model = get_peft_model(model, config)
print_trainable_parameters(lora_model)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=lora_model)

In [ ]:
from utils.dataset import EnJaDatasetMaker
dataset = EnJaDatasetMaker.load_dataset(f"{SOURCE_LANG}-{TARGET_LANG}-{DATASET_NAME}")
train_data = dataset["train"].remove_columns(["source", "target"])
valid_data = dataset["valid"].remove_columns(["source", "target"])

In [ ]:
from utils.metric import SacreBleu
compute_metrics = SacreBleu.get_mBART_metric(tokenizer=tokenizer, target_language=TARGET_LANG)

In [ ]:
def set_decoder_configuration(gc: GenerationConfig):
    gc.no_repeat_ngram_size = 4
    gc.length_penalty = 1.0
    gc.num_beams = 3
    #gen_config.max_new_tokens = MAX_LENGHT
    gc.max_length = 256
    gc.min_length = 0
    gc.early_stopping = True
    # pad token is set to eos since in GPT2 pad does not exist
    gc.pad_token_id = tokenizer.pad_token_id
    gc.bos_token_id = tokenizer.bos_token_id
    gc.eos_token_id = tokenizer.eos_token_id
    return gc

gen_config = GenerationConfig()
gen_config = set_decoder_configuration(gen_config)

In [ ]:
train_args = Seq2SeqTrainingArguments(
    report_to="wandb",
    run_name=f"{SOURCE_LANG}-{TARGET_LANG}-{DATASET_NAME}",
    num_train_epochs=3,

    logging_strategy="steps",
    logging_steps=1,

    evaluation_strategy="steps",
    eval_steps=2500, # 1250, 2500
    prediction_loss_only=False,
    predict_with_generate=True,
    generation_config=gen_config,

    output_dir=f"./.ckp/{SOURCE_LANG}-{TARGET_LANG}-{DATASET_NAME}/",
    save_strategy="steps",
    save_steps=2500, # 1250, 2500
    save_total_limit=100,
    load_best_model_at_end=True, # defaults to metric: "loss"
    metric_for_best_model="eval_score",
    greater_is_better=True,

    optim="adamw_torch",
    warmup_steps=875, # 400, 875
    learning_rate=3e-5, # 3e-5, 5e-5
    bf16=True,
    
    group_by_length=True,
    length_column_name="length",

    # torch_compile=True,
    label_smoothing_factor=0.2,
    
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
)

In [ ]:
trainer = Seq2SeqTrainer(
    lora_model,
    args=train_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=valid_data,
    compute_metrics=compute_metrics
)

lora_model.train()
trainer.train(resume_from_checkpoint=RESUME)